# Introduction

In this notebook we are going to explore the information of the table "Spotify-Top-Songs" of the Database "Spotify-Songs" that we create for the project. We will also document the development of the project.

## Description of the project

For this project, we're going to make a Power BI dashboard. The dashboard will show the most popular songs on Spotify in each country. To develop this project, we're going to upload the dataset to a PostgreSQL database. We'll also use this notebook to preview the data and detect any issues, and then clean the table. After the data is cleaned, we will connect the database to Power BI to create the dashboard.

## Data Description

The data proportioned has the following columns of data: 

* `spotify_id`: Unique identifier for the song in Spotify
* `name`: The title of the song
* `artist`: Name(s) of the artist(s) associated with the song
* `daily_rank`: Daily rank of the song in the top 50 list (0 to 50)
* `daily_movement`: Change in rankings compared to the previous day (-49 to 50)
* `weekly_movement`: Change in rankings compared to the previous week (-49 to 50) 
* `country`: ISO code of the country of the TOP 50 Playlist
* `snapshot_date`: Date on which the data was collected from Spotify 
* `popularity`: Measure of the song's current popularity on Spotify (0 to 100)
* `is_explicit`: Whether the song contains explicit lyrics
* `duration_ms`: The duration of the song in milliseconds 
* `album_name`: Name of the album of the song
* `album_release_date`: Date of the release of the album of the song
* `danceability`
* `energy`
* `key`
* `loudness`
* `mode`
* `speechiness`
* `acousticness`
* `instrumentalness`
* `liveness`
* `valence`
* `tempo`
* `time_signature`

## Procedure

We are going to segment this project in different sections:

* Data Exploration

* Data Cleaning

* Dashboard Creation

Having clarified the above, let's get started.

# Data Exploration

In this section we are going to load the data to a database in PostgreSQL, we are going to connect to that database using Python and explore the data for errors and anomalies.

## Data Load To PostgreSQL

When we load the data to our database, we assign this type of data to the columns: 

* `spotify_id`: Character(30)
* `name`: Text
* `artist`: Text
* `daily_rank`: Integer
* `daily_movement`: Integer
* `weekly_movement`: Integer
* `country`: Character(10)
* `snapshot_date`: Date
* `popularity`: Integer
* `is_explicit`: Boolean
* `duration_ms`: Integer
* `album_name`: Text
* `album_release_date`: Date 
* `danceability`: Double Precision
* `energy`: Double Precision
* `key`: Integer
* `loudness`: Double Precision
* `mode`: Integer
* `speechiness`: Double Precision
* `acousticness`: Double Precision
* `instrumentalness`: Double Precision
* `liveness`: Double Precision
* `valence`: Double Precision
* `tempo`: Double Precision
* `time_signature`: Double Precision

But we couldn't assign the Date type to the column `album_release_date`, let's review why:

In [1]:
# Importing libraries
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# Connection to the data base
db_config = {'user': 'postgres',
                'pwd': '0123456789',
                'host': 'localhost',
                'port': '5432',
                'db': 'Spotify-Songs'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], db_config['pwd'], db_config['host'], db_config['port'], db_config['db'])

engine = create_engine(connection_string)

In [ ]:
# Obtaining the data of the table "spotify_top_songs"
query = """SELECT *
FROM "spotify_top_songs";
"""

df = pd.io.sql.read_sql(query, con = engine)

df.head(5)

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2ctjDCCg1wHoQSjIJ8p6U4,Candy,Plan B,47,-5,-1,SV,2023-12-16,75,False,...,11,-3.735,0,0.1470,0.2520,0.000000,0.5170,0.542,95.981,4
1,0DWdj2oZMBFSzRsi2Cvfzf,TQG,"KAROL G, Shakira",48,1,-10,SV,2023-12-16,89,True,...,4,-3.547,0,0.2770,0.6730,0.000000,0.0936,0.607,179.974,4
2,7JbMsR4rZh6J77LNafur8U,¿por Que Te Demoras?,Plan B,49,1,1,SV,2023-12-16,63,False,...,2,-5.923,0,0.0769,0.0386,0.000000,0.0547,0.941,96.018,4
3,69Ej1xrGjOcHvIMtMKxK0G,Dile,Don Omar,50,-5,-5,SV,2023-12-16,84,False,...,4,-7.501,0,0.1410,0.1840,0.000132,0.0420,0.714,94.001,4
4,06qMRF18gwbOYYbnP2du6i,Last Christmas - Single Version,Wham!,1,0,0,SK,2023-12-16,88,False,...,2,-8.228,1,0.0278,0.2120,0.000004,0.1560,0.935,107.732,4


In [6]:
# Describing the data
display(df.info())
display(df.describe())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728382 entries, 0 to 1728381
Data columns (total 25 columns):
 #   Column              Dtype  
---  ------              -----  
 0   spotify_id          object 
 1   name                object 
 2   artists             object 
 3   daily_rank          int64  
 4   daily_movement      int64  
 5   weekly_movement     int64  
 6   country             object 
 7   snapshot_date       object 
 8   popularity          int64  
 9   is_explicit         bool   
 10  duration_ms         int64  
 11  album_name          object 
 12  album_release_date  object 
 13  danceability        float64
 14  energy              float64
 15  key                 int64  
 16  loudness            float64
 17  mode                int64  
 18  speechiness         float64
 19  acousticness        float64
 20  instrumentalness    float64
 21  liveness            float64
 22  valence             float64
 23  tempo               float64
 24  time_signature      int6

None

,daily_rank,daily_movement,weekly_movement,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06,1.728382e+06
mean,2.549208e+01,9.288537e-01,2.802143e+00,7.609990e+01,1.933514e+05,6.793745e-01,6.492625e-01,5.543001e+00,-6.622988e+00,5.380558e-01,9.488872e-02,2.743085e-01,2.041868e-02,1.707527e-01,5.497221e-01,1.222080e+02,3.901194e+00
std,1.442889e+01,7.011687e+00,1.215836e+01,1.574934e+01,4.963848e+04,1.410755e-01,1.666627e-01,3.579551e+00,3.343594e+00,4.985498e-01,9.088160e-02,2.509523e-01,1.043010e-01,1.248310e-01,2.295444e-01,2.807423e+01,4.045418e-01
min,1.000000e+00,-4.900000e+01,-4.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.010000e-05,0.000000e+00,-5.434100e+01,0.000000e+00,0.000000e+00,3.450000e-06,0.000000e+00,1.390000e-02,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.300000e+01,-1.000000e+00,-3.000000e+00,6.500000e+01,1.616550e+05,5.850000e-01,5.510000e-01,2.000000e+00,-7.805000e+00,0.000000e+00,3.850000e-02,6.700000e-02,0.000000e+00,9.610000e-02,3.710000e-01,1.000160e+02,4.000000e+00
50%,2.500000e+01,0.000000e+00,0.000000e+00,8.000000e+01,1.859170e+05,7.010000e-01,6.680000e-01,6.000000e+00,-6.021000e+00,1.000000e+00,5.780000e-02,1.890000e-01,1.280000e-06,1.210000e-01,5.520000e-01,1.199600e+02,4.000000e+00
75%,3.800000e+01,2.000000e+00,5.000000e+00,8.800000e+01,2.184230e+05,7.830000e-01,7.650000e-01,9.000000e+00,-4.710000e+00,1.000000e+00,1.100000e-01,4.370000e-01,8.500000e-05,2.050000e-01,7.350000e-01,1.400820e+02,4.000000e+00
max,5.000000e+01,4.900000e+01,4.900000e+01,1.000000e+02,9.396660e+05,9.880000e-01,9.980000e-01,1.100000e+01,3.233000e+00,1.000000e+00,9.390000e-01,9.960000e-01,9.950000e-01,9.780000e-01,9.920000e-01,2.360890e+02,5.000000e+00


,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2ctjDCCg1wHoQSjIJ8p6U4,Candy,Plan B,47,-5,-1,SV,2023-12-16,75,False,...,11,-3.735,0,0.1470,0.2520,0.000000,0.5170,0.542,95.981,4
1,0DWdj2oZMBFSzRsi2Cvfzf,TQG,"KAROL G, Shakira",48,1,-10,SV,2023-12-16,89,True,...,4,-3.547,0,0.2770,0.6730,0.000000,0.0936,0.607,179.974,4
2,7JbMsR4rZh6J77LNafur8U,¿por Que Te Demoras?,Plan B,49,1,1,SV,2023-12-16,63,False,...,2,-5.923,0,0.0769,0.0386,0.000000,0.0547,0.941,96.018,4
3,69Ej1xrGjOcHvIMtMKxK0G,Dile,Don Omar,50,-5,-5,SV,2023-12-16,84,False,...,4,-7.501,0,0.1410,0.1840,0.000132,0.0420,0.714,94.001,4
4,06qMRF18gwbOYYbnP2du6i,Last Christmas - Single Version,Wham!,1,0,0,SK,2023-12-16,88,False,...,2,-8.228,1,0.0278,0.2120,0.000004,0.1560,0.935,107.732,4


At first instance, we don't see any anomaly but let's review if the all the rows have the same date format.

In [9]:
# Corroborating if the column "album_release_date" have the same format for all it's rows
query = """SELECT album_release_date
FROM "spotify_top_songs"
WHERE album_release_date !~ '^\d{4}-\d{2}-\d{2}$';
"""

pd.io.sql.read_sql(query, con = engine)

,album_release_date
0,
1,
2,
3,
4,
...,...
653,
654,
655,
656,


As we can se, there are 658 rows that doesn't have information, but in the inicial description of the data we didn't find any missing data, the explanation for this is that the rows it isn't empty at all, it contains "". Let's view which songs have this issue.

In [14]:
# Reviewing the songs that have this "album_release_date" issue:
query = """SELECT name, artists, album_name, COUNT(album_release_date)
FROM "spotify_top_songs"
WHERE album_release_date !~ '^\d{4}-\d{2}-\d{2}$'
GROUP BY name, artists, album_name
ORDER BY name DESC;
"""

pd.io.sql.read_sql(query, con = engine)

,name,artists,album_name,count
0,Silent Night - 1999 Remaster,Frank Sinatra,,3
1,Mistletoe And Holly - Remastered 1999,Frank Sinatra,,1
2,Jingle Bells - Remastered 1999,Frank Sinatra,,200
3,Have Yourself A Merry Little Christmas - Remas...,Frank Sinatra,,92
4,Brown Eyed Girl,Van Morrison,,333
5,,,,29


As we can see, there are there are only five songs that have this issue and there is a song that not even have a name, we can get rid of it.